In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mutual_info_score, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

# Data preprocessing

In [3]:
df = pd.read_csv("Data/housing.csv")
df.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY


In [4]:
# Keep only  '<1H OCEAN' and 'INLAND' from ocean_proximity
df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Fill missing values
print('Missing values\n',df.isnull().sum()[4:5])
df.fillna(0, inplace = True) 

# Log transform to median_house_value
df['median_house_value'] = df['median_house_value'].agg(np.log1p)

# Train/Validation/Test split
df_train_large, df_test = train_test_split(df, test_size = 0.2, random_state = 1)
df_train, df_val = train_test_split(df_train_large, train_size = 0.75, random_state = 1)

# Convert DataFrames to dictionary records
train_dict = df_train.to_dict(orient='records')
train_large_dict = df_train_large.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')
test_dict = df_test.to_dict(orient='records')

dv = DictVectorizer(sparse=True)

# Fit and transform
X_train = dv.fit_transform(train_dict)
X_train_large = dv.transform(train_large_dict)
X_val = dv.transform(val_dict)
X_test = dv.transform(test_dict)


Missing values
 total_bedrooms    157
dtype: int64


# Decision Trees

Consider a hypothetical dataset with multiples numerical features $\mathbf{X}_{1},\cdots, \mathbf{X}_{d}$ and a target variable $\mathbf{Y}$. These features and the target variable are represented as column vectors in the dataset, as shown:

$$
\left( \begin{array}{c|ccc|c}
\text{Instance}    &\mathbf{X}_{1}&\cdots & \mathbf{X}_{d}  & \mathbf{Y}\\
\hline
\mathbf{x}_{1} & x_{11}& \cdots&x_{1d}&y_1 \\
\vdots&\vdots&\ddots&\vdots&\vdots\\
\mathbf{x}_{n}&x_{n1}&\cdots&x_{nd}&y_n
\end{array} \right).$$

In this representation, each row vector $ \mathbf{x}_i = ( x_{i1}, \ldots, x_{id})$ constitutes an instance of the dataset, with $d$ feature values. The index $i$ ranges from 1 to $n$, where $n$ is the total number of instances. The dataset can be further decomposed into a feature matrix $\mathbf{X}$ and a target vector $\mathbf{Y}$:

$$\mathbf{X}=
\left( \begin{array}{ccc}
  x_{11}& \cdots&x_{1d} \\
\vdots&\ddots&\vdots&\\
x_{n1}&\cdots&x_{nd}
\end{array} \right) ~~~ \text{and} ~~~ 

\mathbf{Y} = \left( \begin{array}{c}
y_1\\
\vdots\\
y_n
\end{array} \right)
$$

Here, the feature matrix $\mathbf{X}$ is composed of all feature vectors, while the target vector $\mathbf{Y}$ takes discrete values, which could be binary or multi-classes. Each unique class in $\mathcal{Y}$ corresponds to a partition of the dataset, formed by grouping instances $\mathbf{x}_i$ with the same class.


A decision tree is a recursive model that employs partition-based methods to predict the class $\hat{y}_i$ for each instance $\mathbf{x}_i$. The process starts by splitting the dataset into two partitions. These partitions are further divided recursively, until a state is achieved in which the majority of instances  $\mathbf{x}_i$ within a partition belong to the same class.

One important partition-based method employed in most decision tree models, such as CART, is the axis-parallel hyperplane. This approach is commonly used in high-dimensional spaces represented by the dataset's features. The term "hyperplane" refers to the generalization of a geometric plane in a space with more than three dimensions.

The mathematical formulation for such a hyperplane is given by the condition:

$$h(\mathbf{x}) = \mathbf{x} \cdot \mathbf{w} + b\leq 0$$

Here, the $\mathbf{x}$ variable in this function $h(\mathbf{x})$ can be any instance from the dataset. The weight vector $\mathbf{w}$ is restricted a priori to one of the standard basis vectors $\{\mathbf{e}_1,\cdots,\mathbf{e}_j,\cdots \mathbf{e}_d\}$, where $\mathbf{e}_j$ has a value of 1 for the jth dimension and 0 for all other dimensions. This implies that the weights determine the orientation of the hyperplane in one of the basis vector directions, while the bias term translates it along that axis. The base vectors of a $d$-dimensional space are given by:

$$ 
\mathbf{e}_1 = \left( \begin{array}{c}
1\\
\vdots\\
0\\
\vdots\\
0
\end{array} \right),~
\mathbf{e}_i = \left( \begin{array}{c}
0\\
\vdots\\
1\\
\vdots\\
0
\end{array} \right),~~
\mathbf{e}_d = \left( \begin{array}{c}
0\\
\vdots\\
0\\
\vdots\\
1
\end{array} \right)
$$


On the other hand, the inequality $h(\mathbf{x}) \leq 0$ serves a particular purpose: it defines a half-space. Any instance $\mathbf{x}$ for which $h(\mathbf{x}) \leq 0$ lies on one side of the hyperplane, and any instance for which $h(\mathbf{x}) > 0$ lies on the other side. In this way, the hyperplane acts as a decision boundary that partitions the dataset into two partitions based on the sign of $h(\mathbf{x})$.

For a given standard basis vector chosen as the weight $\mathbf{w} = \mathbf{e}_j$, where $j$ can be an integer between 1 and $d$, the decision condition $h(\mathbf{x})$ for some instance $\mathbf{x}_i$ is represented by:

$$h(\mathbf{x}_i) = \mathbf{e}_j \cdot \mathbf{x}_i  + b \leq 0$$

which simplifies to:

$$x_{ij} \leq t$$

Here $t = -b$ represents a specific value within the domain of the feature vector $\mathbf{X}_j$.The split condition for the ith feature will be then the value of the jth element from the row vector $\mathbf{X}_j$. The optimal offset $b$ is chosen to minimize a particular criterion, such as a loss function, for the partitioned datasets.

Upon applying the decision boundary, the dataset $\mathcal{D}$ is divided into two mutually exclusive partitions: $\mathcal{D}_Y$ and $\mathcal{D}_N$. The partition $\mathcal{D}_Y$ includes instances that satisfy the inequality $x_{ij} \leq t$, while $\mathcal{D}_N$ includes those that do not. More formally, for each instance $\mathbf{x}_i$:

- If the $j$-th feature $x_{ij}$ meets the condition $x_{ij} \leq t$, then the instance is allocated to the set

  $$\mathcal{D}_Y = \{\mathbf{x}_i| x_{ij} \leq  t\}$$
  
  which grouped all instances $\mathbf{x}_i$ such that their $j$-th feature $x_{ij}$ is less than or equal to the threshold $t$.

- Otherwise, the instance are allocated into the set

  $$\mathcal{D}_N = \{\mathbf{x}_i| x_{ij} >  t\ \}$$

In this context, $t$ is a pre-selected threshold that serves as the decision boundary that separates the two partitions


Given that $x_{ij}$ is a element from the feature vector $\mathbf{X}_j$, we can turn this more explicitly in the notation. To express this condition in a more generic form that represents the behavior across the entire feature, we can write the inequality $x_{ij} \leq t$ for all $i = 1, 2, \ldots, n$ as

$$\mathbf{X}_j \leq t$$

This generic form implicitly implies that the condition is applicable to each element $x_{ij}$, where $i = 1, 2, \ldots, n $ across the entire dataset $\mathcal{D}$. By using this generic representation, the rules becomes a general principle that applies not just to individual instances, but to the entire feature, providing a view on how the feature $\mathbf{X}_j$ contributes to the partitioning of $\mathcal{D}$.



**Particular case in two dimension**

To illuminate the partition-based methods used in the decision tree model, let's consider a hypothetical scenario involving a synthetically generated dataset. This dataset will have two features, $\mathbf{X}_1$ and $\mathbf{X}_2$, and a multi-class target variable $\mathbf{Y}$ with four possible classes. Mathematically the dataset can be represented as:

$$
\left( \begin{array}{c|cc|c}
\text{Instance}    &\mathbf{X}_{1}& \mathbf{X}_{2}  & \mathbf{Y}\\
\hline
\mathbf{x}_{1} & x_{11}&x_{12}&y_1 \\
\vdots&\vdots&\vdots&\vdots\\
\mathbf{x}_{n}&x_{n1}&x_{n2}&y_n
\end{array} \right).$$

where each instance will be randomly generated to be in one of the four possible classes of the feature vector $\mathbf{Y}$. Then two threshold values are selected to partition the dataset  $\mathcal{D}$ into four distinct regions $\mathcal{D}_1$, $\mathcal{D}_2$  $\mathcal{D}_3$ and $\mathcal{D}_4$ .

<center><img src = "Images/axis_parallel.png" width="600" height="500"/></center>

By observing the plot, we can gain a visual intuition into how partition-based methods would partition the feature space to isolate instances of different classes.  Each region within this space is defined by a set of rules. These rules act as decision conditions, determining the region to which each instance belongs. When the model evaluates a given instance, it follows this set of decision rules:


- $\mathcal{D_1}$: If $\mathbf{X}_1 \leq 3.5$ and $\mathbf{X}_2 > 4.5$, then belongs to  class 4
- $\mathcal{D_2}$: If $\mathbf{X}_1 > 3.5$ and $\mathbf{X}_2 > 4.5$, then belongs to  class 2
- $\mathcal{D}_3$: If $\mathbf{X}_1 \leq 3.5$ and $\mathbf{X}_2 \leq 4.5$, then belongs to  class 1
- $\mathcal{D_4}$: If $\mathbf{X}_1 > 3.5$ and $\mathbf{X}_2 \leq 4.5$, then belongs to  class 3

After established the partitioning of our feature space into distinct regions, we can represent these partitions in a more structured form, using decision tree diagram, which gives the name of the model. A decision tree diagram will represent a series of decisions made on the features of the dataset to reach a conclusion about the class of an instance. 

To those decision rules, we can draw the following diagram which visually can give a interpretation of the decision-making process in classifying an instance based on its feature values. Consider the following diagram of a decision tree:

<center><img src = "Images/decision_tree_diagram.png" width="600" height="400"/></center>



### **Evaluating split points $t$**

For a numerical or categorical attribute split point, defined as $X\leq v$ or $X \in \mathbf{V}$, we require a scoring criterion to effectively separate the different class labels ${c_1, \cdots, c_k}$.

**Information Gain**

Information gain measures the reduction of disorder or uncertainty in a system. The goal is to use entropy as a metric for each partition, favoring a lower entropy if the partition is pure (i.e., most points have the same class label). Conversely, if class labels are mixed with no majority class, a partition has higher entropy.

The entropy of a set of labeled points $\mathbf{D}$ is defined as:

$$H(\mathbf{D}) = - \sum^{k}_{i=1} P(c_i| \mathbf{D}) \log{P(c_i| \mathbf{D})}$$

where $P(x_i| \mathbf{D})$ is the probability of class $c_i$ in $\mathbf{D}$, and $k$ is the number of classes.

When a split point partitions $\mathbf{D}$ into $\mathbf{D}_Y$ and $\mathbf{D}_N$, we define the split entropy as the weighted entropy of each of the resulting partitions:

$$H(\mathbf{D}_Y, \mathbf{D}_N) = \frac{n_Y}{n}H(\mathbf{D}_Y) + \frac{n_N}{n}H(\mathbf{D}_N)$$

where $n = |\mathbf{D}|$ is the number of points in $\mathbf{D}$, and $n_Y$ and $n_N$ are the number of points in $\mathbf{D}_Y$ and $\mathbf{D}_N$, respectively.

The information gain for a given split point, representing the reduction in overall entropy, is defined as:

$$\text{Grain}(\mathbf{D},\mathbf{D}_Y, \mathbf{D}_N) = H(\mathbf{D}) - H(\mathbf{D}_Y, \mathbf{D}_N)$$

A higher information gain corresponds to a greater reduction in entropy, thus signaling a better split point. Therefore, we can score each split point and select the one that provides the highest information gain.


**Gini Index**

The Gini index, another common purity measure for a split point, is defined as:

$$G(\mathbf{D}) = 1 - \sum_{i=1}^{k} P(c_i| \mathbf{D})^2$$

Higher Gini index values signify more disorder, while lower values indicate a higher order with respect to the class labels. The weighted Gini index of a split point is then defined as:

$$G(\mathbf{D}_Y, \mathbf{D}_N) = \frac{n_Y}{n}G(\mathbf{D}_Y) + \frac{n_N}{n}G(\mathbf{D}_N)$$


**CART**

Another useful measure is the CART, defined as:

$$\text{CART}(\mathbf{D}_Y, \mathbf{D}_N) = 2 \frac{n_Y}{n}\frac{n_N}{n} \sum_{i = 1}^k |P(c_i| \mathbf{D}_Y )  -P(c_i| \mathbf{D}_N)$$

This metric maximizes the difference between the class probability mass function for the two partitions; a higher CART value implies a better split point.


$$
\left( \begin{array}{c|cc|c}
\text{Instance}    &\mathbf{X}_{1}& \mathbf{X}_{2}  & \mathbf{Y}\\
\hline
\mathbf{x}_{1} & x_{11}&x_{12}&y_1 \\
\vdots&\vdots&\vdots&\vdots\\
\mathbf{x}_{n}&x_{n1}&x_{n2}&y_n
\end{array} \right).$$